In [1]:
import numpy as np
from scipy import stats
import scipy
import pandas as pd
import pickle
import glob
import seaborn as sns
import networkx as nx
import time

from math import nan
import datetime

import csv
from tqdm import tqdm

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 13})

### Here, we groups stocks by their industry classification to do sector analysis

In [2]:
#wrds = pd.read_csv('/scratch/Debbie_Miori/pricesWRDS.csv')
wrds_ind = pd.read_csv('/scratch/Debbie_Miori/stocks_data/pricesWRDS_industry.csv')
wrds_ind = wrds_ind.dropna()
wrds_ind = wrds_ind[wrds_ind.RET!='C']
wrds_ind.head()

/home/miori/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,PERMNO,date,TICKER,COMNAM,CUSIP,HSICMG,HSICIG,PRC,RET,sprtrn
3870,10026,20190805,JJSF,J & J SNACK FOODS CORP,46603210,20.0,205.0,181.74001,-0.002086,-0.029778
3871,10026,20190806,JJSF,J & J SNACK FOODS CORP,46603210,20.0,205.0,182.07001,0.001816,0.013017
3872,10026,20190807,JJSF,J & J SNACK FOODS CORP,46603210,20.0,205.0,184.87000,0.015379,0.000767
3873,10026,20190808,JJSF,J & J SNACK FOODS CORP,46603210,20.0,205.0,186.14999,0.006924,0.018762
3874,10026,20190809,JJSF,J & J SNACK FOODS CORP,46603210,20.0,205.0,186.42999,0.001504,-0.006617


In [4]:
np.unique(wrds_ind.HSICMG)

array([ 1.,  2.,  7.,  9., 10., 12., 13., 14., 15., 16., 17., 20., 21.,
       22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34.,
       35., 36., 37., 38., 39., 40., 41., 42., 44., 45., 46., 47., 48.,
       49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61.,
       62., 63., 64., 65., 67., 70., 72., 73., 75., 76., 78., 79., 80.,
       82., 83., 86., 87., 89., 94., 96., 97., 99.])

In [8]:
x = wrds_ind.loc[:, ['TICKER', 'HSICMG']]
x = x.drop_duplicates()
x

,TICKER,HSICMG
3870,JJSF,20.0
6232,ELA,50.0
8431,PLXS,36.0
9556,RMCF,20.0
12715,HNGR,48.0
...,...,...
16834518,CORN,62.0
16842953,FN,38.0
16845616,CBOE,62.0
16847054,VLTC,99.0


In [9]:
np.unique(list(x[x.HSICMG==20.].TICKER))

array(['ABEV', 'ADM', 'ANFI', 'BDBD', 'BETR', 'BF', 'BG', 'BGS', 'BH',
       'BRFS', 'BRID', 'BTH', 'CAG', 'CCE', 'CCEP', 'CCH', 'COKE', 'CPB',
       'CREG', 'CZZ', 'DAR', 'DF', 'FARM', 'FTK', 'GIS', 'GLDC', 'GMCR',
       'HAIN', 'HSY', 'IBA', 'JJSF', 'KDP', 'LANC', 'LBCC', 'LPCN', 'LSF',
       'LTEA', 'LW', 'LWAY', 'MDLZ', 'MED', 'MKC', 'MNST', 'MXC', 'OCCH',
       'OMAD', 'PACK', 'PEP', 'PLAG', 'POST', 'PPC', 'PRMW', 'REED',
       'RELV', 'RIBT', 'RMCF', 'ROX', 'RYCE', 'SENEA', 'SENEB', 'SJM',
       'SMBL', 'STZ', 'SXT', 'TAP', 'TR', 'TSN', 'WWAV', 'YVR', 'ZVIA'],
      dtype='<U5')

In [5]:
"""
    https://siccode.com/page/structure-of-sic-codes
"""

agric = [1.,  2.,  7.,  9.]
mining = [10., 12., 13., 14.]
construction = [ 15., 16., 17.]
manufacturing = [20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39.]
transportation_and_public = [40., 41., 42., 44., 45., 46., 47., 48., 49.]
wholesale = [50., 51.]
retail = [52., 53., 54., 55., 56., 57., 58., 59.]
Finance_Insurance_RealEstate = [60., 61., 62., 63., 64., 65., 67.]
Services = [70., 72., 73., 75., 76., 78., 79]
PublicAdministration = [94., 96., 97., 99.]

In [11]:
ag = []
for i in agric:
    ag += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
m = []
for i in mining:
    m += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
c = []
for i in construction:
    c += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
ma = []
for i in manufacturing:
    ma += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
t = []
for i in transportation_and_public:
    t += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
w = []
for i in wholesale:
    w += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
r = []
for i in retail:
    r += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
f = []
for i in Finance_Insurance_RealEstate:
    f += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
s = []
for i in Services:
    s += list(np.unique(list(x[x.HSICMG==i].TICKER)))
    
p = []
for i in PublicAdministration:
    p += list(np.unique(list(x[x.HSICMG==i].TICKER)))

In [13]:
groups = {
    'agric': ag,
    'mining': m,
    'costr': c,
    'manuf': ma,
    'transp': t,
    'wholesale': w,
    'retail': r,
    'fin_ins_RE': f,
    'services': s,
    'pubAdm': p
}

In [16]:
for key in groups.keys():
    print(key, len(groups[key]))

agric 16
mining 352
costr 56
manuf 1227
transp 441
wholesale 149
retail 194
fin_ins_RE 5136
services 597
pubAdm 2897


In [17]:
import pickle
with open('/scratch/Debbie_Miori/groups_stocks.pickle', 'wb') as handle:
    pickle.dump(groups, handle)